In [1]:
%load_ext dotenv
%dotenv

In [6]:

from autogen import ConversableAgent, AssistantAgent, GroupChat, GroupChatManager

from demo.utils import custom_print_received_message
from demo.utils import llm_config
from demo.glam import initialize_glam_agents

#
# Customize the system prompt
#
strategy_agent_system_prompt = """
You're a crypto fund manager that manages an index fund on the Solana blockchain that tracks 50% USDC and 50% WSOL.

Here are some tokens and their mint addresses, you need them when you want to swap tokens for rebalancing:
- WSOL: So11111111111111111111111111111111111111112
- USDC: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v

You should check fund balances and swap between USDC and WSOL if needed to make the fund 50% USDC and 50% WSOL. If the difference between the two token's values is less than 5 USD, then do nothing.

For simplicity, you can assume that SOL and WSOL are interchangeable names of the same token.
"""
strategy_agent = AssistantAgent(name="strategy_agent", llm_config=llm_config, system_message=strategy_agent_system_prompt)

glam_agents = initialize_glam_agents(llm_config)

groupchat = GroupChat(agents=[glam_agents[0], glam_agents[1], strategy_agent], messages=[], max_round=15)
groupchat_manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
chat_result = strategy_agent.initiate_chat(
    groupchat_manager, message="""Hi there!"""
)